In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Activation
from keras.models import load_model

In [ ]:
df = pd.read_csv('/content/BTC-USD (1).csv')
df.head(5)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-01-01,14112.200195,14112.200195,13154.700195,13657.200195,13657.200195,10291200000
1,2018-01-02,13625.000000,15444.599609,13163.599609,14982.099609,14982.099609,16846600192
2,2018-01-03,14978.200195,15572.799805,14844.500000,15201.000000,15201.000000,16871900160
3,2018-01-04,15270.700195,15739.700195,14522.200195,15599.200195,15599.200195,21783199744
4,2018-01-05,15477.200195,17705.199219,15202.799805,17429.500000,17429.500000,23840899072


In [ ]:
df=df.drop(columns=['Date','Open','High','Volume','Low','Close'])
df

,Adj Close
0,13657.200195
1,14982.099609
2,15201.000000
3,15599.200195
4,17429.500000
...,...
1822,16552.572266
1823,16642.341797
1824,16602.585938
1825,16547.496094


In [ ]:
def lagrange_interpolation(x, y, x_interp):

    def lagrange_basis(j):
        p = np.poly1d([1])
        for m in range(len(x)):
            if m != j:
                p *= np.poly1d([1, -x[m]]) / (x[j] - x[m])
        return p


    result = np.zeros_like(x_interp, dtype=float)
    for j in range(len(x)):
        result += y[j] * lagrange_basis(j)(x_interp)

    return result

In [ ]:
def rate_of_return(prices):
    return prices.pct_change()

In [ ]:
def price_var(prices, window_size):
    return prices.rolling(window=window_size).var()


In [ ]:
df['Adj Close']

0       13657.200195
1       14982.099609
2       15201.000000
3       15599.200195
4       17429.500000
            ...     
1822    16552.572266
1823    16642.341797
1824    16602.585938
1825    16547.496094
1826    16625.080078
Name: Adj Close, Length: 1827, dtype: float64

In [ ]:
def moving_average(prices, window_size):
    return prices.rolling(window=window_size).mean()


In [ ]:
def psychological_index(returns):
    positive_returns = returns[returns > 0].count()
    total_returns = len(returns)
    return positive_returns / total_returns

In [ ]:
def Boll_index(prices, window_size):
    rolling_mean = prices.rolling(window=window_size).mean()
    rolling_std = prices.rolling(window=window_size).std()
    upper_band = rolling_mean + 2 * rolling_std
    middle_band = rolling_mean
    lower_band = rolling_mean - 2 * rolling_std
    return upper_band, middle_band, lower_band


In [ ]:
def calculate_rsi(prices, window_size):
    diff = prices.diff()
    gains = diff.where(diff > 0, 0)
    losses = -diff.where(diff < 0, 0)
    avg_gains = gains.rolling(window=window_size, min_periods=1).mean()
    avg_losses = losses.rolling(window=window_size, min_periods=1).mean()
    rs = avg_gains / avg_losses
    rsi = 100 - (100 / (1 + rs))

    return rsi

In [ ]:
def xy(data,steps=24):
  x_train = []
  y_train = []

  for i in range(steps, data.shape[0]-steps):
      x_train.append(data.iloc[i-steps:i, :])
      y_train.append(data.iloc[i, :])

  x_train, y_train = np.array(x_train), np.array(y_train)
  return x_train,y_train

In [ ]:
xt,yt= xy(df[['Adj Close']],20)
# df['Train adj close']=xt
# df['Eng Adj close']=yt
xt.shape

(1787, 20, 1)

In [ ]:
con=pd.DataFrame(xt.reshape(1787,20))

In [ ]:
con

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,13657.200195,14982.099609,15201.000000,15599.200195,17429.500000,17527.000000,16477.599609,15170.099609,14595.400391,14973.299805,13405.799805,13980.599609,14360.200195,13772.000000,13819.799805,11490.500000,11188.599609,11474.900391,11607.400391,12899.200195
1,14982.099609,15201.000000,15599.200195,17429.500000,17527.000000,16477.599609,15170.099609,14595.400391,14973.299805,13405.799805,13980.599609,14360.200195,13772.000000,13819.799805,11490.500000,11188.599609,11474.900391,11607.400391,12899.200195,11600.099609
2,15201.000000,15599.200195,17429.500000,17527.000000,16477.599609,15170.099609,14595.400391,14973.299805,13405.799805,13980.599609,14360.200195,13772.000000,13819.799805,11490.500000,11188.599609,11474.900391,11607.400391,12899.200195,11600.099609,10931.400391
3,15599.200195,17429.500000,17527.000000,16477.599609,15170.099609,14595.400391,14973.299805,13405.799805,13980.599609,14360.200195,13772.000000,13819.799805,11490.500000,11188.599609,11474.900391,11607.400391,12899.200195,11600.099609,10931.400391,10868.400391
4,17429.500000,17527.000000,16477.599609,15170.099609,14595.400391,14973.299805,13405.799805,13980.599609,14360.200195,13772.000000,13819.799805,11490.500000,11188.599609,11474.900391,11607.400391,12899.200195,11600.099609,10931.400391,10868.400391,11359.400391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1782,16697.777344,16711.546875,16291.832031,15787.284180,16189.769531,16610.707031,16604.464844,16521.841797,16464.281250,16444.626953,16217.322266,16444.982422,17168.566406,16967.132813,17088.660156,16908.236328,17130.486328,16974.826172,17089.503906,16848.126953
1783,16711.546875,16291.832031,15787.284180,16189.769531,16610.707031,16604.464844,16521.841797,16464.281250,16444.626953,16217.322266,16444.982422,17168.566406,16967.132813,17088.660156,16908.236328,17130.486328,16974.826172,17089.503906,16848.126953,17233.474609
1784,16291.832031,15787.284180,16189.769531,16610.707031,16604.464844,16521.841797,16464.281250,16444.626953,16217.322266,16444.982422,17168.566406,16967.132813,17088.660156,16908.236328,17130.486328,16974.826172,17089.503906,16848.126953,17233.474609,17133.152344
1785,15787.284180,16189.769531,16610.707031,16604.464844,16521.841797,16464.281250,16444.626953,16217.322266,16444.982422,17168.566406,16967.132813,17088.660156,16908.236328,17130.486328,16974.826172,17089.503906,16848.126953,17233.474609,17133.152344,17128.724609


In [ ]:
ma=moving_average(df['Adj Close'],10)
ma2=moving_average(df['Adj Close'],30)
ror=rate_of_return(df['Adj Close'])
var = price_var(df['Adj Close'], 10)
var2= price_var(df['Adj Close'], 20)
upper,middle,lower = Boll_index(df['Adj Close'], 10)

In [ ]:
rsi=calculate_rsi(df['Adj Close'],14)

In [ ]:
psycho = psychological_index(ror)

In [ ]:
df['Moving Average(10)']=ma
df

,Adj Close,Moving Average(10)
0,13657.200195,NaN
1,14982.099609,NaN
2,15201.000000,NaN
3,15599.200195,NaN
4,17429.500000,NaN
...,...,...
1822,16552.572266,16767.010742
1823,16642.341797,16787.276953
1824,16602.585938,16756.905078
1825,16547.496094,16729.901172


In [ ]:
df['Moving Average(30)']=ma2
df['Rate of Return']=ror
df['Price Variance(10)']=var
df['Price Variance(20)']=var2
df['Boll Upper']=upper
df['Boll Middle']=middle
df['Boll Lower']=lower
df.head(25)


,Adj Close,Moving Average(10),Moving Average(30),Rate of Return,Price Variance(10),Price Variance(20),Boll Upper,Boll Middle,Boll Lower
0,13657.200195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14982.099609,NaN,NaN,0.097011,NaN,NaN,NaN,NaN,NaN
2,15201.000000,NaN,NaN,0.014611,NaN,NaN,NaN,NaN,NaN
3,15599.200195,NaN,NaN,0.026196,NaN,NaN,NaN,NaN,NaN
4,17429.500000,NaN,NaN,0.117333,NaN,NaN,NaN,NaN,NaN
5,17527.000000,NaN,NaN,0.005594,NaN,NaN,NaN,NaN,NaN
6,16477.599609,NaN,NaN,-0.059873,NaN,NaN,NaN,NaN,NaN
7,15170.099609,NaN,NaN,-0.079350,NaN,NaN,NaN,NaN,NaN
8,14595.400391,NaN,NaN,-0.037884,NaN,NaN,NaN,NaN,NaN
9,14973.299805,15561.239941,NaN,0.025892,1.524202e+06,NaN,18030.411496,15561.239941,13092.068387


In [ ]:
df['RSI']=rsi

In [ ]:
con[20:]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
20,11600.099609,10931.400391,10868.400391,11359.400391,11259.400391,11171.400391,11440.700195,11786.299805,11296.400391,10106.299805,10221.099609,9170.540039,8830.750000,9174.910156,8277.009766,6955.270020,7754.000000,7621.299805,8265.589844,8736.980469
21,10931.400391,10868.400391,11359.400391,11259.400391,11171.400391,11440.700195,11786.299805,11296.400391,10106.299805,10221.099609,9170.540039,8830.750000,9174.910156,8277.009766,6955.270020,7754.000000,7621.299805,8265.589844,8736.980469,8621.900391
22,10868.400391,11359.400391,11259.400391,11171.400391,11440.700195,11786.299805,11296.400391,10106.299805,10221.099609,9170.540039,8830.750000,9174.910156,8277.009766,6955.270020,7754.000000,7621.299805,8265.589844,8736.980469,8621.900391,8129.970215
23,11359.400391,11259.400391,11171.400391,11440.700195,11786.299805,11296.400391,10106.299805,10221.099609,9170.540039,8830.750000,9174.910156,8277.009766,6955.270020,7754.000000,7621.299805,8265.589844,8736.980469,8621.900391,8129.970215,8926.570313
24,11259.400391,11171.400391,11440.700195,11786.299805,11296.400391,10106.299805,10221.099609,9170.540039,8830.750000,9174.910156,8277.009766,6955.270020,7754.000000,7621.299805,8265.589844,8736.980469,8621.900391,8129.970215,8926.570313,8598.309570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1782,16697.777344,16711.546875,16291.832031,15787.284180,16189.769531,16610.707031,16604.464844,16521.841797,16464.281250,16444.626953,16217.322266,16444.982422,17168.566406,16967.132813,17088.660156,16908.236328,17130.486328,16974.826172,17089.503906,16848.126953
1783,16711.546875,16291.832031,15787.284180,16189.769531,16610.707031,16604.464844,16521.841797,16464.281250,16444.626953,16217.322266,16444.982422,17168.566406,16967.132813,17088.660156,16908.236328,17130.486328,16974.826172,17089.503906,16848.126953,17233.474609
1784,16291.832031,15787.284180,16189.769531,16610.707031,16604.464844,16521.841797,16464.281250,16444.626953,16217.322266,16444.982422,17168.566406,16967.132813,17088.660156,16908.236328,17130.486328,16974.826172,17089.503906,16848.126953,17233.474609,17133.152344
1785,15787.284180,16189.769531,16610.707031,16604.464844,16521.841797,16464.281250,16444.626953,16217.322266,16444.982422,17168.566406,16967.132813,17088.660156,16908.236328,17130.486328,16974.826172,17089.503906,16848.126953,17233.474609,17133.152344,17128.724609


In [ ]:
data=df[20:-20]


In [ ]:
data.reset_index(drop=True, inplace=True)
data

,Adj Close,Moving Average(10),Moving Average(30),Rate of Return,Price Variance(10),Price Variance(20),Boll Upper,Boll Middle,Boll Lower,RSI
0,11600.099609,12619.329980,NaN,-0.100712,1.602843e+06,3.773441e+06,15151.398530,12619.329980,10087.261431,27.947627
1,10931.400391,12314.410059,NaN,-0.057646,1.610208e+06,4.208229e+06,14852.289731,12314.410059,9776.530386,29.660947
2,10868.400391,11965.230078,NaN,-0.005763,1.242033e+06,4.542142e+06,14194.160551,11965.230078,9736.299606,31.192726
3,11359.400391,11723.970117,NaN,0.045177,8.554270e+05,4.574835e+06,13573.756079,11723.970117,9874.184155,31.969271
4,11259.400391,11467.930176,NaN,-0.008803,3.185113e+05,3.891485e+06,12596.666278,11467.930176,10339.194074,37.453828
...,...,...,...,...,...,...,...,...,...,...
1782,17233.474609,16985.399609,16696.987174,0.022872,5.054845e+04,1.540848e+05,17435.059256,16985.399609,16535.739962,60.621275
1783,17133.152344,17054.216602,16738.732910,-0.005821,1.526204e+04,1.641557e+05,17301.295892,17054.216602,16807.137311,60.261078
1784,17128.724609,17050.232422,16723.464681,-0.000258,1.440836e+04,1.626927e+05,17290.302064,17050.232422,16810.162780,61.355481
1785,17104.193359,17063.938476,16725.794694,-0.001432,1.375588e+04,1.162553e+05,17298.509388,17063.938476,16829.367565,61.253374


In [ ]:
con.reset_index(drop=True,inplace=True)
con

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,13657.200195,14982.099609,15201.000000,15599.200195,17429.500000,17527.000000,16477.599609,15170.099609,14595.400391,14973.299805,13405.799805,13980.599609,14360.200195,13772.000000,13819.799805,11490.500000,11188.599609,11474.900391,11607.400391,12899.200195
1,14982.099609,15201.000000,15599.200195,17429.500000,17527.000000,16477.599609,15170.099609,14595.400391,14973.299805,13405.799805,13980.599609,14360.200195,13772.000000,13819.799805,11490.500000,11188.599609,11474.900391,11607.400391,12899.200195,11600.099609
2,15201.000000,15599.200195,17429.500000,17527.000000,16477.599609,15170.099609,14595.400391,14973.299805,13405.799805,13980.599609,14360.200195,13772.000000,13819.799805,11490.500000,11188.599609,11474.900391,11607.400391,12899.200195,11600.099609,10931.400391
3,15599.200195,17429.500000,17527.000000,16477.599609,15170.099609,14595.400391,14973.299805,13405.799805,13980.599609,14360.200195,13772.000000,13819.799805,11490.500000,11188.599609,11474.900391,11607.400391,12899.200195,11600.099609,10931.400391,10868.400391
4,17429.500000,17527.000000,16477.599609,15170.099609,14595.400391,14973.299805,13405.799805,13980.599609,14360.200195,13772.000000,13819.799805,11490.500000,11188.599609,11474.900391,11607.400391,12899.200195,11600.099609,10931.400391,10868.400391,11359.400391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1782,16697.777344,16711.546875,16291.832031,15787.284180,16189.769531,16610.707031,16604.464844,16521.841797,16464.281250,16444.626953,16217.322266,16444.982422,17168.566406,16967.132813,17088.660156,16908.236328,17130.486328,16974.826172,17089.503906,16848.126953
1783,16711.546875,16291.832031,15787.284180,16189.769531,16610.707031,16604.464844,16521.841797,16464.281250,16444.626953,16217.322266,16444.982422,17168.566406,16967.132813,17088.660156,16908.236328,17130.486328,16974.826172,17089.503906,16848.126953,17233.474609
1784,16291.832031,15787.284180,16189.769531,16610.707031,16604.464844,16521.841797,16464.281250,16444.626953,16217.322266,16444.982422,17168.566406,16967.132813,17088.660156,16908.236328,17130.486328,16974.826172,17089.503906,16848.126953,17233.474609,17133.152344
1785,15787.284180,16189.769531,16610.707031,16604.464844,16521.841797,16464.281250,16444.626953,16217.322266,16444.982422,17168.566406,16967.132813,17088.660156,16908.236328,17130.486328,16974.826172,17089.503906,16848.126953,17233.474609,17133.152344,17128.724609


In [ ]:
df=pd.concat([data,con],axis=1)
df.head(2)

,Adj Close,Moving Average(10),Moving Average(30),Rate of Return,Price Variance(10),Price Variance(20),Boll Upper,Boll Middle,Boll Lower,RSI,...,10,11,12,13,14,15,16,17,18,19
0,11600.099609,12619.329980,NaN,-0.100712,1.602843e+06,3.773441e+06,15151.398530,12619.329980,10087.261431,27.947627,...,13405.799805,13980.599609,14360.200195,13772.000000,13819.799805,11490.500000,11188.599609,11474.900391,11607.400391,12899.200195
1,10931.400391,12314.410059,NaN,-0.057646,1.610208e+06,4.208229e+06,14852.289731,12314.410059,9776.530386,29.660947,...,13980.599609,14360.200195,13772.000000,13819.799805,11490.500000,11188.599609,11474.900391,11607.400391,12899.200195,11600.099609


In [ ]:
df.head(5)

,Adj Close,Moving Average(10),Moving Average(30),Rate of Return,Price Variance(10),Price Variance(20),Boll Upper,Boll Middle,Boll Lower,RSI,...,10,11,12,13,14,15,16,17,18,19
0,11600.099609,12619.329980,NaN,-0.100712,1.602843e+06,3.773441e+06,15151.398530,12619.329980,10087.261431,27.947627,...,13405.799805,13980.599609,14360.200195,13772.000000,13819.799805,11490.500000,11188.599609,11474.900391,11607.400391,12899.200195
1,10931.400391,12314.410059,NaN,-0.057646,1.610208e+06,4.208229e+06,14852.289731,12314.410059,9776.530386,29.660947,...,13980.599609,14360.200195,13772.000000,13819.799805,11490.500000,11188.599609,11474.900391,11607.400391,12899.200195,11600.099609
2,10868.400391,11965.230078,NaN,-0.005763,1.242033e+06,4.542142e+06,14194.160551,11965.230078,9736.299606,31.192726,...,14360.200195,13772.000000,13819.799805,11490.500000,11188.599609,11474.900391,11607.400391,12899.200195,11600.099609,10931.400391
3,11359.400391,11723.970117,NaN,0.045177,8.554270e+05,4.574835e+06,13573.756079,11723.970117,9874.184155,31.969271,...,13772.000000,13819.799805,11490.500000,11188.599609,11474.900391,11607.400391,12899.200195,11600.099609,10931.400391,10868.400391
4,11259.400391,11467.930176,NaN,-0.008803,3.185113e+05,3.891485e+06,12596.666278,11467.930176,10339.194074,37.453828,...,13819.799805,11490.500000,11188.599609,11474.900391,11607.400391,12899.200195,11600.099609,10931.400391,10868.400391,11359.400391


In [ ]:
df

,Adj Close,Moving Average(10),Moving Average(30),Rate of Return,Price Variance(10),Price Variance(20),Boll Upper,Boll Middle,Boll Lower,RSI,...,10,11,12,13,14,15,16,17,18,19
20,11600.099609,12619.329980,NaN,-0.100712,1.602843e+06,3.773441e+06,15151.398530,12619.329980,10087.261431,27.947627,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,10931.400391,12314.410059,NaN,-0.057646,1.610208e+06,4.208229e+06,14852.289731,12314.410059,9776.530386,29.660947,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,10868.400391,11965.230078,NaN,-0.005763,1.242033e+06,4.542142e+06,14194.160551,11965.230078,9736.299606,31.192726,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,11359.400391,11723.970117,NaN,0.045177,8.554270e+05,4.574835e+06,13573.756079,11723.970117,9874.184155,31.969271,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,11259.400391,11467.930176,NaN,-0.008803,3.185113e+05,3.891485e+06,12596.666278,11467.930176,10339.194074,37.453828,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1782,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,16217.322266,16444.982422,17168.566406,16967.132813,17088.660156,16908.236328,17130.486328,16974.826172,17089.503906,16848.126953
1783,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,16444.982422,17168.566406,16967.132813,17088.660156,16908.236328,17130.486328,16974.826172,17089.503906,16848.126953,17233.474609
1784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,17168.566406,16967.132813,17088.660156,16908.236328,17130.486328,16974.826172,17089.503906,16848.126953,17233.474609,17133.152344
1785,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,16967.132813,17088.660156,16908.236328,17130.486328,16974.826172,17089.503906,16848.126953,17233.474609,17133.152344,17128.724609


In [ ]:
def remove_rows_with_nan(df):

    while df.isnull().any(axis=1).any():
        rows_with_nan = df[df.isnull().any(axis=1)]
        df = df.dropna()
        if not df.isnull().any(axis=1).any():
            break

    return df

In [ ]:
df=remove_rows_with_nan(df)

In [ ]:
df

,Adj Close,Moving Average(10),Moving Average(30),Rate of Return,Price Variance(10),Price Variance(20),Boll Upper,Boll Middle,Boll Lower,RSI,...,10,11,12,13,14,15,16,17,18,19
9,10106.299805,11181.980176,13181.040039,-0.105352,2.195387e+05,1.512811e+06,12119.079396,11181.980176,10244.880956,40.137091,...,12899.200195,11600.099609,10931.400391,10868.400391,11359.400391,11259.400391,11171.400391,11440.700195,11786.299805,11296.400391
10,10221.099609,11044.080176,13066.503353,0.011359,2.815725e+05,1.545621e+06,12105.348209,11044.080176,9982.812143,42.917380,...,11600.099609,10931.400391,10868.400391,11359.400391,11259.400391,11171.400391,11440.700195,11786.299805,11296.400391,10106.299805
11,9170.540039,10867.994141,12872.784701,-0.102783,6.357272e+05,1.614422e+06,12462.644242,10867.994141,9273.344039,34.828472,...,10931.400391,10868.400391,11359.400391,11259.400391,11171.400391,11440.700195,11786.299805,11296.400391,10106.299805,10221.099609
12,8830.750000,10664.229102,12660.443034,-0.037052,1.050745e+06,1.531475e+06,12714.346360,10664.229102,8614.111843,32.204720,...,10868.400391,11359.400391,11259.400391,11171.400391,11440.700195,11786.299805,11296.400391,10106.299805,10221.099609,9170.540039
13,9174.910156,10445.780078,12446.300033,0.038973,1.190479e+06,1.399053e+06,12627.961941,10445.780078,8263.598215,22.831301,...,11359.400391,11259.400391,11171.400391,11440.700195,11786.299805,11296.400391,10106.299805,10221.099609,9170.540039,8830.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1782,17233.474609,16985.399609,16696.987174,0.022872,5.054845e+04,1.540848e+05,17435.059256,16985.399609,16535.739962,60.621275,...,16217.322266,16444.982422,17168.566406,16967.132813,17088.660156,16908.236328,17130.486328,16974.826172,17089.503906,16848.126953
1783,17133.152344,17054.216602,16738.732910,-0.005821,1.526204e+04,1.641557e+05,17301.295892,17054.216602,16807.137311,60.261078,...,16444.982422,17168.566406,16967.132813,17088.660156,16908.236328,17130.486328,16974.826172,17089.503906,16848.126953,17233.474609
1784,17128.724609,17050.232422,16723.464681,-0.000258,1.440836e+04,1.626927e+05,17290.302064,17050.232422,16810.162780,61.355481,...,17168.566406,16967.132813,17088.660156,16908.236328,17130.486328,16974.826172,17089.503906,16848.126953,17233.474609,17133.152344
1785,17104.193359,17063.938476,16725.794694,-0.001432,1.375588e+04,1.162553e+05,17298.509388,17063.938476,16829.367565,61.253374,...,16967.132813,17088.660156,16908.236328,17130.486328,16974.826172,17089.503906,16848.126953,17233.474609,17133.152344,17128.724609


In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df

,Adj Close,Moving Average(10),Moving Average(30),Rate of Return,Price Variance(10),Price Variance(20),Boll Upper,Boll Middle,Boll Lower,RSI,...,10,11,12,13,14,15,16,17,18,19
0,10106.299805,11181.980176,13181.040039,-0.105352,2.195387e+05,1.512811e+06,12119.079396,11181.980176,10244.880956,40.137091,...,12899.200195,11600.099609,10931.400391,10868.400391,11359.400391,11259.400391,11171.400391,11440.700195,11786.299805,11296.400391
1,10221.099609,11044.080176,13066.503353,0.011359,2.815725e+05,1.545621e+06,12105.348209,11044.080176,9982.812143,42.917380,...,11600.099609,10931.400391,10868.400391,11359.400391,11259.400391,11171.400391,11440.700195,11786.299805,11296.400391,10106.299805
2,9170.540039,10867.994141,12872.784701,-0.102783,6.357272e+05,1.614422e+06,12462.644242,10867.994141,9273.344039,34.828472,...,10931.400391,10868.400391,11359.400391,11259.400391,11171.400391,11440.700195,11786.299805,11296.400391,10106.299805,10221.099609
3,8830.750000,10664.229102,12660.443034,-0.037052,1.050745e+06,1.531475e+06,12714.346360,10664.229102,8614.111843,32.204720,...,10868.400391,11359.400391,11259.400391,11171.400391,11440.700195,11786.299805,11296.400391,10106.299805,10221.099609,9170.540039
4,9174.910156,10445.780078,12446.300033,0.038973,1.190479e+06,1.399053e+06,12627.961941,10445.780078,8263.598215,22.831301,...,11359.400391,11259.400391,11171.400391,11440.700195,11786.299805,11296.400391,10106.299805,10221.099609,9170.540039,8830.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1773,17233.474609,16985.399609,16696.987174,0.022872,5.054845e+04,1.540848e+05,17435.059256,16985.399609,16535.739962,60.621275,...,16217.322266,16444.982422,17168.566406,16967.132813,17088.660156,16908.236328,17130.486328,16974.826172,17089.503906,16848.126953
1774,17133.152344,17054.216602,16738.732910,-0.005821,1.526204e+04,1.641557e+05,17301.295892,17054.216602,16807.137311,60.261078,...,16444.982422,17168.566406,16967.132813,17088.660156,16908.236328,17130.486328,16974.826172,17089.503906,16848.126953,17233.474609
1775,17128.724609,17050.232422,16723.464681,-0.000258,1.440836e+04,1.626927e+05,17290.302064,17050.232422,16810.162780,61.355481,...,17168.566406,16967.132813,17088.660156,16908.236328,17130.486328,16974.826172,17089.503906,16848.126953,17233.474609,17133.152344
1776,17104.193359,17063.938476,16725.794694,-0.001432,1.375588e+04,1.162553e+05,17298.509388,17063.938476,16829.367565,61.253374,...,16967.132813,17088.660156,16908.236328,17130.486328,16974.826172,17089.503906,16848.126953,17233.474609,17133.152344,17128.724609


In [ ]:

pip install arch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.0/983.0 kB 18.1 MB/s eta 0:00:00


In [ ]:
from arch import arch_model
returns = df['Rate of Return']

model = arch_model(returns, vol='Garch', p=1, q=1)
result = model.fit()

stochastic_disturbance = result.resid
conditional_variance = result.conditional_volatility


Iteration:      1,   Func. Count:      6,   Neg. LLF: 89932960620.35068
Iteration:      2,   Func. Count:     18,   Neg. LLF: 3640.4240251231513
Iteration:      3,   Func. Count:     27,   Neg. LLF: 15068.927158097762
Iteration:      4,   Func. Count:     37,   Neg. LLF: 66159233.13356546
Iteration:      5,   Func. Count:     43,   Neg. LLF: 1081609235.658527
Iteration:      6,   Func. Count:     50,   Neg. LLF: -3292.031532141128
Iteration:      7,   Func. Count:     56,   Neg. LLF: -3294.174984113199
Iteration:      8,   Func. Count:     62,   Neg. LLF: -3346.4239718920026
Iteration:      9,   Func. Count:     68,   Neg. LLF: -3355.852187515623
Iteration:     10,   Func. Count:     74,   Neg. LLF: -3356.5550620842414
Iteration:     11,   Func. Count:     79,   Neg. LLF: -3356.5556422304444
Iteration:     12,   Func. Count:     84,   Neg. LLF: -3356.555651940311
Iteration:     13,   Func. Count:     88,   Neg. LLF: -3356.5556519388592
Optimization terminated successfully    (Exit mode

/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.001451. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(


In [ ]:
conditional_variance

0       0.071352
1       0.072500
2       0.068019
3       0.069428
4       0.065923
          ...   
1773    0.029927
1774    0.030082
1775    0.029743
1776    0.029393
1777    0.029095
Name: cond_vol, Length: 1778, dtype: float64

In [ ]:
stochastic_disturbance

0      -0.106697
1       0.010014
2      -0.104129
3      -0.038398
4       0.037628
          ...   
1773    0.021527
1774   -0.007167
1775   -0.001604
1776   -0.002777
1777    0.004632
Name: resid, Length: 1778, dtype: float64

In [ ]:
df['Stochastic Disturbance'] = stochastic_disturbance
df['Conditional Variance'] = conditional_variance

In [ ]:
df

,Adj Close,Moving Average(10),Moving Average(30),Rate of Return,Price Variance(10),Price Variance(20),Boll Upper,Boll Middle,Boll Lower,RSI,...,12,13,14,15,16,17,18,19,Stochastic Disturbance,Conditional Variance
0,10106.299805,11181.980176,13181.040039,-0.105352,2.195387e+05,1.512811e+06,12119.079396,11181.980176,10244.880956,40.137091,...,10931.400391,10868.400391,11359.400391,11259.400391,11171.400391,11440.700195,11786.299805,11296.400391,-0.106697,0.071352
1,10221.099609,11044.080176,13066.503353,0.011359,2.815725e+05,1.545621e+06,12105.348209,11044.080176,9982.812143,42.917380,...,10868.400391,11359.400391,11259.400391,11171.400391,11440.700195,11786.299805,11296.400391,10106.299805,0.010014,0.072500
2,9170.540039,10867.994141,12872.784701,-0.102783,6.357272e+05,1.614422e+06,12462.644242,10867.994141,9273.344039,34.828472,...,11359.400391,11259.400391,11171.400391,11440.700195,11786.299805,11296.400391,10106.299805,10221.099609,-0.104129,0.068019
3,8830.750000,10664.229102,12660.443034,-0.037052,1.050745e+06,1.531475e+06,12714.346360,10664.229102,8614.111843,32.204720,...,11259.400391,11171.400391,11440.700195,11786.299805,11296.400391,10106.299805,10221.099609,9170.540039,-0.038398,0.069428
4,9174.910156,10445.780078,12446.300033,0.038973,1.190479e+06,1.399053e+06,12627.961941,10445.780078,8263.598215,22.831301,...,11171.400391,11440.700195,11786.299805,11296.400391,10106.299805,10221.099609,9170.540039,8830.750000,0.037628,0.065923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1773,17233.474609,16985.399609,16696.987174,0.022872,5.054845e+04,1.540848e+05,17435.059256,16985.399609,16535.739962,60.621275,...,17168.566406,16967.132813,17088.660156,16908.236328,17130.486328,16974.826172,17089.503906,16848.126953,0.021527,0.029927
1774,17133.152344,17054.216602,16738.732910,-0.005821,1.526204e+04,1.641557e+05,17301.295892,17054.216602,16807.137311,60.261078,...,16967.132813,17088.660156,16908.236328,17130.486328,16974.826172,17089.503906,16848.126953,17233.474609,-0.007167,0.030082
1775,17128.724609,17050.232422,16723.464681,-0.000258,1.440836e+04,1.626927e+05,17290.302064,17050.232422,16810.162780,61.355481,...,17088.660156,16908.236328,17130.486328,16974.826172,17089.503906,16848.126953,17233.474609,17133.152344,-0.001604,0.029743
1776,17104.193359,17063.938476,16725.794694,-0.001432,1.375588e+04,1.162553e+05,17298.509388,17063.938476,16829.367565,61.253374,...,16908.236328,17130.486328,16974.826172,17089.503906,16848.126953,17233.474609,17133.152344,17128.724609,-0.002777,0.029393


In [ ]:
n_train_rows = int(df.shape[0]*.9)-1
train = df.iloc[:n_train_rows, :]
test = df.iloc[n_train_rows:, :]
print(train.shape)
print(test.shape)

(1599, 32)
(179, 32)


In [ ]:
X_train= train.drop(columns='Adj Close')
y_train= train['Adj Close'].copy()
X_test= test.drop(columns='Adj Close')
y_test= test['Adj Close'].copy()

X_train


,Moving Average(10),Moving Average(30),Rate of Return,Price Variance(10),Price Variance(20),Boll Upper,Boll Middle,Boll Lower,RSI,0,...,12,13,14,15,16,17,18,19,Stochastic Disturbance,Conditional Variance
0,11181.980176,13181.040039,-0.105352,2.195387e+05,1.512811e+06,12119.079396,11181.980176,10244.880956,40.137091,14973.299805,...,10931.400391,10868.400391,11359.400391,11259.400391,11171.400391,11440.700195,11786.299805,11296.400391,-0.106697,0.071352
1,11044.080176,13066.503353,0.011359,2.815725e+05,1.545621e+06,12105.348209,11044.080176,9982.812143,42.917380,13405.799805,...,10868.400391,11359.400391,11259.400391,11171.400391,11440.700195,11786.299805,11296.400391,10106.299805,0.010014,0.072500
2,10867.994141,12872.784701,-0.102783,6.357272e+05,1.614422e+06,12462.644242,10867.994141,9273.344039,34.828472,13980.599609,...,11359.400391,11259.400391,11171.400391,11440.700195,11786.299805,11296.400391,10106.299805,10221.099609,-0.104129,0.068019
3,10664.229102,12660.443034,-0.037052,1.050745e+06,1.531475e+06,12714.346360,10664.229102,8614.111843,32.204720,14360.200195,...,11259.400391,11171.400391,11440.700195,11786.299805,11296.400391,10106.299805,10221.099609,9170.540039,-0.038398,0.069428
4,10445.780078,12446.300033,0.038973,1.190479e+06,1.399053e+06,12627.961941,10445.780078,8263.598215,22.831301,13772.000000,...,11171.400391,11440.700195,11786.299805,11296.400391,10106.299805,10221.099609,9170.540039,8830.750000,0.037628,0.065923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594,29650.373438,29814.932227,-0.056351,1.797866e+06,1.439807e+06,32332.063793,29650.373438,26968.683083,38.861682,29098.910156,...,29832.914063,29906.662109,31370.671875,31155.478516,30214.355469,30111.998047,29083.804688,28360.810547,-0.057696,0.034325
1595,28928.673242,29561.136328,-0.159747,6.919746e+06,4.105198e+06,34189.755230,28928.673242,23667.591254,17.098089,29655.585938,...,29906.662109,31370.671875,31155.478516,30214.355469,30111.998047,29083.804688,28360.810547,26762.648438,-0.161093,0.036656
1596,28166.061133,29257.858984,-0.012478,1.120311e+07,6.703846e+06,34860.269706,28166.061133,21471.852560,16.378158,29562.361328,...,31370.671875,31155.478516,30214.355469,30111.998047,29083.804688,28360.810547,26762.648438,22487.388672,-0.013823,0.055072
1597,27432.678907,29014.856380,0.016484,1.374488e+07,8.796405e+06,34847.495954,27432.678907,20017.861859,21.387375,29267.224609,...,31155.478516,30214.355469,30111.998047,29083.804688,28360.810547,26762.648438,22487.388672,22206.792969,0.015138,0.052178


In [ ]:
print(X_train.head(10))
print(y_train.head(10))

   Moving Average(10)  Moving Average(30)  Rate of Return  Price Variance(10)  \
0        11181.980176        13181.040039       -0.105352        2.195387e+05   
1        11044.080176        13066.503353        0.011359        2.815725e+05   
2        10867.994141        12872.784701       -0.102783        6.357272e+05   
3        10664.229102        12660.443034       -0.037052        1.050745e+06   
4        10445.780078        12446.300033        0.038973        1.190479e+06   
5        10147.541016        12141.217025       -0.097865        1.540715e+06   
6         9725.927979        11788.826025       -0.159688        2.359019e+06   
7         9357.257959        11498.039372        0.114838        2.313339e+06   
8         8940.757959        11246.412712       -0.017114        1.799848e+06   
9         8637.676904        11035.419027        0.084538        1.131872e+06   

   Price Variance(20)    Boll Upper   Boll Middle    Boll Lower        RSI  \
0        1.512811e+06  12119.0

In [ ]:
y_train.shape

(1599,)

In [ ]:
sc = MinMaxScaler(feature_range = (0, 1))
X_training_set_scaled = sc.fit_transform(X_train.values)
# y_training_set_scaled = sc.fit_transform(y_train.values)
X_test_set_scaled = sc.fit_transform(X_test.values)

In [ ]:
from keras.layers import Input,Dense, Dropout, Embedding, LSTM, Bidirectional, Concatenate, Attention
from keras.models import Model
from keras.datasets import imdb

In [ ]:
X_train.shape

(1599, 31)

In [ ]:
y_train

0       10106.299805
1       10221.099609
2        9170.540039
3        8830.750000
4        9174.910156
            ...     
1594    26762.648438
1595    22487.388672
1596    22206.792969
1597    22572.839844
1598    20381.650391
Name: Adj Close, Length: 1599, dtype: float64

In [ ]:
X_training_set_scaled

array([[0.12699387, 0.1623135 , 0.47632722, ..., 0.12528572, 0.47632722,
        0.57591406],
       [0.12474434, 0.16037955, 0.68505353, ..., 0.10678581, 0.68505353,
        0.59105552],
       [0.1218719 , 0.15710862, 0.48092122, ..., 0.10857035, 0.48092122,
        0.5319948 ],
       ...,
       [0.40405056, 0.43377006, 0.64242326, ..., 0.29924774, 0.64242326,
        0.36136785],
       [0.39208709, 0.42966697, 0.69421785, ..., 0.29488593, 0.69421785,
        0.323228  ],
       [0.374161  , 0.42401376, 0.4911356 , ..., 0.30057606, 0.4911356 ,
        0.28902957]])

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional, Layer,Concatenate,Flatten,Attention,Input
from keras.callbacks import EarlyStopping
from keras.optimizers import RMSprop
from keras.models import Model
import tensorflow as tf

In [ ]:
input_layer = Input(shape=(X_train.shape[1], 1))
blstm_layer = Bidirectional(LSTM(units = 69,activation='relu',return_sequences=True))(input_layer)
lstm_layer2 = Bidirectional(LSTM(units = 30,return_sequences=True))(blstm_layer)
lstm_layer3 = Bidirectional(LSTM(units = 15,return_sequences=True))(lstm_layer2)

attention =Attention()([lstm_layer3,lstm_layer3])
merged = Concatenate()([lstm_layer3, attention])
flatten = Flatten()(merged)
output_layer = Dense(1)(flatten)

model = Model(inputs=input_layer,outputs = output_layer)
from keras.layers import Dropout

dropout1 = Dropout(0.2)(lstm_layer2)
dropout2 = Dropout(0.2)(lstm_layer3)
 



In [ ]:
model.compile(optimizer='rmsprop',loss='mean_squared_error')
early_stopping = EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)
model.fit(X_train, y_train,epochs=12,validation_data=(X_test,y_test),callbacks=[early_stopping])
loss = model.evaluate(X_test,y_test)
print(f"Test loss:{loss}")


In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
mape = mean_absolute_percentage_error(y_test,predictions)*100
print(mape)

In [ ]:
model = Sequential()
epochs = 300

# # model.add(Bidirectional(LSTM(50,activation='relu'), input_shape = (X_train.shape[1],1)))
# model.add(LSTM(units=50,activation='relu', input_shape = (X_training_set_scaled.shape[1],1)))
# # model.add(Dropout(0.2))
# # model.add(LSTM(units=35, return_sequences = False))
# # model.add(Dropout(0.2))
# # model.add(Dense(units=25))
# # model.build((X_train.shape[1],1))
# # model.add(Dropout(0.2))
# model.add(Dense(units=25))
# # model.add(Dropout(0.2))
# model.add(Dense(units=1))
# model.compile(loss='mse', optimizer='adam')
# model.fit(X_training_set_scaled, y_train, batch_size = 32, epochs = epochs)
# model.summary

Epoch 1/300
50/50 [==============================] - 3s 14ms/step - loss: 605447552.0000
Epoch 2/300
50/50 [==============================] - 1s 13ms/step - loss: 241541600.0000
Epoch 3/300
50/50 [==============================] - 1s 14ms/step - loss: 109898552.0000
Epoch 4/300
50/50 [==============================] - 1s 13ms/step - loss: 36296328.0000
Epoch 5/300
50/50 [==============================] - 1s 13ms/step - loss: 18981632.0000
Epoch 6/300
50/50 [==============================] - 1s 14ms/step - loss: 17670876.0000
Epoch 7/300
42/50 [========================>.....] - ETA: 0s - loss: 16972982.0000

KeyboardInterrupt: 